In [1]:
import requests
import json
import time
from bs4 import BeautifulSoup
import pandas as pd
import datetime

In [2]:
username = ""
password = ""

contest_id = "rco-contest-2018-qual"
problem_id = "rco_contest_2018_qual_a"

contest_start = datetime.datetime(2023,2,9,19,0,0)
contest_end = datetime.datetime(2023,2,15,19,0,0)

In [3]:
personal = {
 'yamake': 'Hypertext Assasins',
 'motoshira': 'Homy Jabirus',
 'cplusplusonly': 'Hypertext Assasins',
 'yas_yasyu': 'へびつかい',
 'edomondo': 'ネバーギブアップの掟',
 'shu8Cream': 'Homy Jabirus',
 'hotpepsi': 'Hypertext Assasins',
 'kokatsu': 'チョコ零個焼きなます！',
 'donphan': 'へびつかい',
 'jaku': 'Homy Jabirus',
 '2bit': 'TheCodeGang5',
 'ssaattoo': 'チョコ零個焼きなます！',
 'ainem': '運営',
 'michirakara': 'へびつかい',
 'yu_w': 'Homy Jabirus',
 'tardigrade': 'ネバーギブアップの掟',
 'r3yohei': 'チョコ零個焼きなます！',
 'june193': 'ネバーギブアップの掟',
 'meowmeowcats': 'TheCodeGang5',
 'tnodino': 'へびつかい',
 'brthyyjp': 'Homy Jabirus',
 'gootara': 'へびつかい',
 'midnq': 'へびつかい',
 'ntk_ta01': 'チョコ零個焼きなます！',
 'platinum': 'Hypertext Assasins',
 'keroru': 'ネバーギブアップの掟',
 'highjump': 'ネバーギブアップの掟',
 'ococonomy1': 'チョコ零個焼きなます！',
 'jamuojisan': 'ネバーギブアップの掟',
 'utprg': 'TheCodeGang5',
 'tmikada': 'TheCodeGang5'
}

In [4]:
df=pd.DataFrame.from_dict(personal, orient="index", columns=["チーム名"])
df.to_csv("personal_info.csv")

In [5]:
personal = pd.read_csv("personal_info.csv", index_col=0)
personal.head()

,チーム名
yamake,Hypertext Assasins
motoshira,Homy Jabirus
cplusplusonly,Hypertext Assasins
yas_yasyu,へびつかい
edomondo,ネバーギブアップの掟


In [6]:
# User Agentの設定の設定
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.4.1 Safari/605.1.15"
header = {
    'User-Agent': user_agent
}

login_url = "https://atcoder.jp/login?continue=https%3A%2F%2Fatcoder.jp%2F"
session = requests.session()
response = session.get(login_url, headers=header)

# BeautifulSoupオブジェクト作成(token取得の為)
bs = BeautifulSoup(response.text, 'html.parser')
# tokenの取得
csrf_token = bs.find(attrs={'name':'csrf_token'}).get('value')

login_data = {
   'username': username,
   'password': password,
   'csrf_token': csrf_token,
}

# cookieの取得
response_cookie = response.cookies

login = session.post(login_url, data=login_data, cookies=response_cookie)
time.sleep(2) # 少し時間を置いてみる

In [7]:
standings_url = "https://atcoder.jp/contests/" + contest_id + "/standings/json"
standings_json = session.get(standings_url).json()

In [8]:
def make_datetime(s: str) -> datetime:
  ymd, hms = s.split(" ")
  year, month, day = map(int,ymd.split("-"))
  hour, minute, second = hms.split(":")
  second = int(second.split("+")[0])

  res = datetime.datetime(year=year, month=month, day=day, hour=int(hour), minute=int(minute), second=int(second))

  return res

In [9]:
records = {
    "name": [],
    "team": [],
    "lang": [],
    "score": [],
    "submit": [],
    "elapsed_time": [],
    #"argo_rate": [],
    #"heuristic_rate": [],
    #"performance": []
}

submission_url = "https://atcoder.jp/contests/" + contest_id +"/submissions?desc=true&f.LanguageName=&f.Status=AC&f.Task=" + problem_id + "&f.User=&orderBy=created&page="
page=1

not_in_record = set()

while True:
  submissions = session.get(submission_url + str(page))
  soup = BeautifulSoup(submissions.text, "html5lib")
  tr = soup.select("tbody > tr")
  if len(tr)==0:
    break
  for row in tr:
    submit = make_datetime(row.contents[1].text)
    if contest_start < submit < contest_end:
      name = row.contents[5].text.rstrip() if " " in row.contents[5].text else row.contents[5].text
      lang = row.contents[7].text
      score = int(row.find(name="td", attrs={"class": "submission-score"}).text)
      elapsed_time = row.contents[14].text
      if name in personal.index:
        tname = personal.at[name, "チーム名"]
        records["name"].append(name)
        records["team"].append(tname)
        records["lang"].append(lang)
        records["score"].append(score)
        records["submit"].append(submit)
        records["elapsed_time"].append(elapsed_time)
        #records["argo_rate"].append(a_rate)
        #records["heuristic_rate"].append(h_rate)
        #records["performance"].append(perf)
      else:
        not_in_record.add(name)
    elif submit < contest_start:
      break
  else:
    page += 1
    continue
  break

if not_in_record: print("バチャ期間中に提出があったが、参加者に登録されていない：", not_in_record)

バチャ期間中に提出があったが、参加者に登録されていない： {'komori3', 'Klavis', 'terry_u16', 'rhoo', 'thunder', 'yuuki_n'}


In [10]:
df = pd.DataFrame(data = records)
df

,name,team,lang,score,submit,elapsed_time
0,june193,ネバーギブアップの掟,Python (3.8.2),257158,2023-02-15 18:59:02,3838 ms
1,yu_w,Homy Jabirus,PyPy3 (7.3.0),113354,2023-02-15 18:59:01,1243 ms
2,gootara,へびつかい,Rust (1.42.0),261211,2023-02-15 18:55:04,3692 ms
3,r3yohei,チョコ零個焼きなます！,Rust (1.42.0),254253,2023-02-15 18:55:01,3998 ms
4,brthyyjp,Homy Jabirus,PyPy3 (7.3.0),270211,2023-02-15 18:53:01,3874 ms
...,...,...,...,...,...,...
475,utprg,TheCodeGang5,PyPy3 (7.3.0),6287,2023-02-09 19:56:14,180 ms
476,ococonomy1,チョコ零個焼きなます！,C++ (GCC 9.2.1),5893,2023-02-09 19:47:01,10 ms
477,utprg,TheCodeGang5,PyPy3 (7.3.0),10033,2023-02-09 19:30:51,1933 ms
478,tnodino,へびつかい,PyPy3 (7.3.0),181,2023-02-09 19:20:26,60 ms


In [11]:
df.to_csv("submissions.csv")